<a href="https://colab.research.google.com/github/OmamaSajid/FB-SentiAnalyzer/blob/main/Facebooksentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"omamasajid","key":"bcdfe380e086c9c208ad9642ab909ab1"}'}

In [ ]:
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/


In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d mortena/fb-sentiment

Dataset URL: https://www.kaggle.com/datasets/mortena/fb-sentiment
License(s): unknown
  0% 0.00/48.1k [00:00<?, ?B/s]
100% 48.1k/48.1k [00:00<00:00, 224MB/s]


In [ ]:
!unzip -q fb-sentiment.zip -d fb-sentiment

In [ ]:
!ls fb-sentiment

fb_sentiment.csv


In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [ ]:
facebook_data=pd.read_csv('/content/fb-sentiment/fb_sentiment.csv',encoding='ISO-8859-1')


In [ ]:
facebook_data.shape

(1000, 3)

In [ ]:
facebook_data.head()

,Unnamed: 0,FBPost,Label
0,0,Drug Runners and a U.S. Senator have somethin...,O
1,1,"Heres a single, to add, to Kindle. Just read t...",O
2,2,If you tire of Non-Fiction.. Check out http://...,O
3,3,Ghost of Round Island is supposedly nonfiction.,O
4,4,Why is Barnes and Nobles version of the Kindle...,N


In [ ]:
facebook_data.isnull().sum()

,0
Unnamed: 0,0
FBPost,0
Label,0


In [ ]:
facebook_data['Label'].value_counts()

,count
Label,
1,641
0,280
2,79


changung thr labels so positive=1 neutral is 0 and negative is 2

In [ ]:
facebook_data.replace({'Label':{'P':1}},inplace=True)
facebook_data.replace({'Label':{'O':0}},inplace=True)
facebook_data.replace({'Label':{'N':2}},inplace=True)

/tmp/ipython-input-18-639890408.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  facebook_data.replace({'Label':{'N':2}},inplace=True)


In [ ]:
port_stem=PorterStemmer()

In [ ]:
def stemming(content):
  stemmed_content=re.sub('[^a-zA-Z]',' ',content)
  stemmed_content=stemmed_content.lower()
  stemmed_content=stemmed_content.split()
  stemmed_content=[port_stem.stem(word)for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content=' '.join(stemmed_content)
  return stemmed_content

In [ ]:
facebook_data['stemmed_content']=facebook_data['FBPost'].apply(stemming)

In [ ]:
facebook_data.head()

,Unnamed: 0,FBPost,Label,stemmed_content
0,0,Drug Runners and a U.S. Senator have somethin...,0,drug runner u senat someth murder http www ama...
1,1,"Heres a single, to add, to Kindle. Just read t...",0,here singl add kindl read th centuri stori gho...
2,2,If you tire of Non-Fiction.. Check out http://...,0,tire non fiction check http www amazon com ref...
3,3,Ghost of Round Island is supposedly nonfiction.,0,ghost round island supposedli nonfict
4,4,Why is Barnes and Nobles version of the Kindle...,2,barn nobl version kindl much expens kindl


In [ ]:
print(facebook_data['stemmed_content'])

0      drug runner u senat someth murder http www ama...
1      here singl add kindl read th centuri stori gho...
2      tire non fiction check http www amazon com ref...
3                  ghost round island supposedli nonfict
4              barn nobl version kindl much expens kindl
                             ...                        
995                 like youth orient think widen appeal
996    think point commerci even border close mani st...
997               kindl great product could happier mine
998    develop way share book big drawback love kindl...
999                                           love kindl
Name: stemmed_content, Length: 1000, dtype: object


In [ ]:
X=facebook_data['stemmed_content'].values
Y=facebook_data['Label'].values

In [ ]:
print(X)

['drug runner u senat someth murder http www amazon com circumstanti evid get florida bozarth ebook dp b fpz ref pd rhf p state attorney know get ypur copi today'
 'here singl add kindl read th centuri stori ghost round island man french american indian dog sled transport woman across ice mackinac island cheboygan ghost'
 'tire non fiction check http www amazon com ref nb sb noss url search alia dap field keyword daniel lee zwissler x'
 'ghost round island supposedli nonfict'
 'barn nobl version kindl much expens kindl'
 'maria mean nook care book buy kindl piec electron vice versa love kindl peopl swear nook like color screen want eread reader dont need color kindl batteri last longer unit isnt heavi make differ read hour'
 'kindl awesom mine great' 'love mine'
 'meh think singl bad idea big name author alreadi domin market huge factor let compet price point indi author albeit give less content money'
 'daugjhter love'
 'sure got updat dont locat number unless press menu button also d

In [ ]:
print(Y)

[0 0 0 0 2 1 1 1 2 1 2 1 1 1 2 1 1 1 1 1 0 1 1 2 1 0 0 2 0 2 0 0 1 0 1 1 0
 0 1 0 1 1 2 0 1 1 1 0 1 0 2 1 1 0 0 0 1 0 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 0
 1 1 0 1 0 1 1 1 1 1 2 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1
 1 1 0 1 1 1 0 1 1 1 0 0 1 0 1 1 1 1 1 1 1 0 0 1 1 0 0 2 1 1 1 0 1 0 1 1 1
 0 0 0 1 1 1 0 1 1 0 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 0 1 1 1 1 0 1 0
 0 1 1 1 1 1 1 1 1 0 0 2 2 1 1 2 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1
 0 1 1 1 1 0 2 1 1 1 0 1 1 1 1 1 1 1 2 2 1 1 0 0 1 1 1 1 1 1 0 1 1 1 0 1 1
 1 0 1 1 1 1 1 1 0 0 1 0 1 0 1 1 2 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 0 0 1 2 2
 0 0 1 1 1 0 1 2 1 1 0 1 2 0 0 1 0 1 1 0 2 0 0 0 0 0 1 0 1 0 1 1 0 2 0 0 0
 1 0 1 1 1 1 0 2 0 1 2 0 0 0 0 0 1 1 0 1 1 1 0 0 1 1 2 0 0 0 0 2 1 2 1 0 0
 1 1 1 0 2 0 1 0 0 1 1 0 0 0 0 2 0 0 0 0 1 0 0 1 1 0 1 1 1 1 2 0 1 1 1 0 1
 0 1 1 1 0 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 0 2 1 1 2 1 2 1 1 0 1 0 0 1 1 0
 1 1 1 1 1 1 0 1 0 1 1 1 2 0 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 0 1
 0 0 1 1 0 1 2 0 1 1 1 0 

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)


In [ ]:
print(X.shape,X_train.shape,X_test.shape)

(1000,) (800,) (200,)


In [ ]:
print(Y.shape,Y_train.shape,Y_test.shape)

(1000,) (800,) (200,)


In [ ]:
vectorizer=TfidfVectorizer()
X_train=vectorizer.fit_transform(X_train)
X_test=vectorizer.transform(X_test)

In [ ]:
print(X_train)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 7914 stored elements and shape (800, 1920)>
  Coords	Values
  (0, 1025)	0.21434478499408002
  (0, 939)	0.08452687488579816
  (0, 505)	0.27004699048694264
  (0, 1366)	0.4078278002073048
  (0, 1658)	0.25773127369250026
  (0, 75)	0.32170764764700144
  (0, 1372)	0.21713562054722707
  (0, 750)	0.1851436380873044
  (0, 171)	0.2574936189931686
  (0, 961)	0.30100126660431653
  (0, 1422)	0.261515835863276
  (0, 928)	0.32170764764700144
  (0, 384)	0.35710543984249704
  (1, 1025)	0.18608007084449915
  (1, 939)	0.14676136736844658
  (1, 1684)	0.3245705866793151
  (1, 1613)	0.6200309986238589
  (1, 143)	0.3613176634042188
  (1, 719)	0.4115389027767165
  (1, 558)	0.39260168224836667
  (2, 939)	0.10177369446643217
  (2, 750)	0.22292024969051463
  (2, 171)	0.15501624153189691
  (2, 732)	0.3873487087198773
  (2, 841)	0.3021080301177654
  :	:
  (799, 557)	0.14300508020062172
  (799, 1790)	0.1377672537379041
  (799, 1240)	0.14115098472325008
 

In [ ]:
model=LogisticRegression(max_iter=1000)

In [ ]:
model.fit(X_train,Y_train)

LogisticRegression(max_iter=1000)

In [ ]:
X_train_pred=model.predict(X_train)
train_acc=accuracy_score(Y_train,X_train_pred)

In [ ]:
print('Accuracy score on training data:',train_acc)

Accuracy score on training data: 0.885


In [ ]:
X_test_pred=model.predict(X_test)
train_accr=accuracy_score(Y_test,X_test_pred)
print('Accuracy score on test data:',train_accr)

Accuracy score on test data: 0.76


In [ ]:
import pickle

In [ ]:
filename='trained_fb_model.sav'
pickle.dump(model,open(filename,'wb'))

In [ ]:
loaded_model=pickle.load(open('/content/trained_fb_model.sav','rb'))

In [ ]:
x_new=X_test[10]
print(Y_test[10])

0


In [ ]:
prediction=model.predict(x_new)
print(prediction)
if (prediction[0]==0):
  print("Neutral comment")
elif(prediction[0]==1):
  print("positive comment")
elif(prediction[0]==2):
  print("negative comment")




[0]
Neutral comment


In [ ]:
my_comment = "I really enjoy using this app, it's so smooth and helpful!"
processed_comment = stemming(my_comment)
comment_vector = vectorizer.transform([processed_comment])

In [ ]:
prediction = loaded_model.predict(comment_vector)
print(prediction)
if (prediction[0]==0):
  print("Neutral comment")
elif(prediction[0]==1):
  print("positive comment")
elif(prediction[0]==2):
  print("negative comment")

[1]
positive comment


In [ ]:
my_comment = "The app was updated today.."
processed_comment = stemming(my_comment)
comment_vector = vectorizer.transform([processed_comment])
prediction = loaded_model.predict(comment_vector)
print(prediction)
if (prediction[0]==0):
  print("Neutral comment")
elif(prediction[0]==1):
  print("positive comment")
elif(prediction[0]==2):
  print("negative comment")

[0]
Neutral comment
